<a href="https://colab.research.google.com/github/GabeAspir/Patent-Prior-Art-Finder/blob/main/4_SimilarityTechniques/4Patent_Prior_Art_Finder_Gabe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal -- To try some similarity techniques!

In [1]:
import pandas as pd
import re
url = 'https://drive.google.com/file/d/18DdQd4ZPbcvOeZ6x2KRJHmvpGarnw9Qx/view?usp=sharing'
file_id = url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
dataframe = pd.read_csv(dwn_url)

We now have a data set (of 10 patents) containing three alike patents - liquid Laundry detergent Patents.

In [2]:
dataframe

,Publication_Number,Abstract,Description,Claim
0,US-7365259-B2,A keyboard apparatus is including a plurality ...,BACKGROUND OF THE INVENTION \n 1. Field of t...,1. A keyboard apparatus comprising:\n a suppor...
1,US-7556524-B2,"An easy-pull type swivel plug includes a body,...",BACKGROUND OF THE INVENTION \n 1. Field of t...,"1. An easy-pull type swivel plug, comprising:\..."
2,US-7338315-B2,The invention relates to a closure device comp...,FIELD OF THE INVENTION \n The invention rela...,1. A closure device comprising:\n a wall havin...
3,US-6843642-B2,An air compressor with shock-absorption rubber...,BACKGROUND OF THE INVENTION \n (a) Field of ...,1. An air compressor with shock-absorption rub...
4,US-9433212-B2,Provided is a novel plant growth regulator. Th...,TECHNICAL FIELD \n The present invention r...,The invention claimed is: \n \n 1. A...
5,US-5536436-A,A liquid laundry detergent composition contain...,FIELD OF THE INVENTION \n The present inve...,What is claimed is: \n \n 1. A heavy...
6,US-2015111807-A1,A liquid laundry detergent composition compris...,FIELD OF THE INVENTION \n The present inve...,What is claimed is: \n \n 1 . A li...
7,US-7605322-B2,As a player inputs a performance of a music pi...,TECHNICAL FIELD \n The present invention rel...,1. An apparatus for automatically starting an ...
8,US-7205268-B2,A low-foaming aqueous liquid laundry detergent...,FIELD OF THE INVENTION \n The present inve...,1. A low-foaming aqueous liquid laundry deterg...
9,US-6910186-B2,A new class of avatars (“organizational avatar...,CROSS-REFERENCE TO APPENDICES ATTACHED HERETO ...,"1. A method of communicating between users, th..."


Here's code for tokenizing a string with these restrictions - <br/> For each row, <br/>
1. split on whitespace, punctuation, apostrophes, etc.
2. Set everything to lowercase
3. Replace numbers with some unique token (e.g., “_NUM_”) -- the number 1, 100, and 1000 should all map to the same token.
4. Remove all one-letter “words”


In [3]:
def tokenize(string):
    lowercasedString = string.lower()
    #To split based on white space and random charactars
    stringArray = re.split('\W+', lowercasedString)
    #Will substitute numbers for _NUM_
    stringArray[:]= [re.sub(r"[0-9]+","_NUM_",s) for s in stringArray]
    #Will filter out 1 letter words like "I" and "a"
    stringArray = list(filter (lambda s: len (s) > 1, stringArray))
    #Will return a List/Array
    return stringArray

Now, we have to take the abstract of each patent,<br/>
and create a new column in the dataframe to put the tokenized<br/> abstract data -<br/>
This is actually really simple using Pandas!

In [4]:
dataframe['Tokenized_Abstract'] = dataframe['Abstract'].apply(tokenize)

In [5]:
dataframe

,Publication_Number,Abstract,Description,Claim,Tokenized_Abstract
0,US-7365259-B2,A keyboard apparatus is including a plurality ...,BACKGROUND OF THE INVENTION \n 1. Field of t...,1. A keyboard apparatus comprising:\n a suppor...,"[keyboard, apparatus, is, including, plurality..."
1,US-7556524-B2,"An easy-pull type swivel plug includes a body,...",BACKGROUND OF THE INVENTION \n 1. Field of t...,"1. An easy-pull type swivel plug, comprising:\...","[an, easy, pull, type, swivel, plug, includes,..."
2,US-7338315-B2,The invention relates to a closure device comp...,FIELD OF THE INVENTION \n The invention rela...,1. A closure device comprising:\n a wall havin...,"[the, invention, relates, to, closure, device,..."
3,US-6843642-B2,An air compressor with shock-absorption rubber...,BACKGROUND OF THE INVENTION \n (a) Field of ...,1. An air compressor with shock-absorption rub...,"[an, air, compressor, with, shock, absorption,..."
4,US-9433212-B2,Provided is a novel plant growth regulator. Th...,TECHNICAL FIELD \n The present invention r...,The invention claimed is: \n \n 1. A...,"[provided, is, novel, plant, growth, regulator..."
5,US-5536436-A,A liquid laundry detergent composition contain...,FIELD OF THE INVENTION \n The present inve...,What is claimed is: \n \n 1. A heavy...,"[liquid, laundry, detergent, composition, cont..."
6,US-2015111807-A1,A liquid laundry detergent composition compris...,FIELD OF THE INVENTION \n The present inve...,What is claimed is: \n \n 1 . A li...,"[liquid, laundry, detergent, composition, comp..."
7,US-7605322-B2,As a player inputs a performance of a music pi...,TECHNICAL FIELD \n The present invention rel...,1. An apparatus for automatically starting an ...,"[as, player, inputs, performance, of, music, p..."
8,US-7205268-B2,A low-foaming aqueous liquid laundry detergent...,FIELD OF THE INVENTION \n The present inve...,1. A low-foaming aqueous liquid laundry deterg...,"[low, foaming, aqueous, liquid, laundry, deter..."
9,US-6910186-B2,A new class of avatars (“organizational avatar...,CROSS-REFERENCE TO APPENDICES ATTACHED HERETO ...,"1. A method of communicating between users, th...","[new, class, of, avatars, organizational, avat..."


Function for BoW

In [6]:
def bagOfWordize(tokenized_abstract):
  #Create a Dictionary
  wordFrequency = {}
  for word in tokenized_abstract:
    if word not in wordFrequency.keys():
      wordFrequency[word] = 1
    else:
      wordFrequency[word] += 1
  return wordFrequency

In [7]:
dataframe['Bag_Of_Words_Abstract'] = dataframe['Tokenized_Abstract'].apply(bagOfWordize)

In [8]:
dataframe

,Publication_Number,Abstract,Description,Claim,Tokenized_Abstract,Bag_Of_Words_Abstract
0,US-7365259-B2,A keyboard apparatus is including a plurality ...,BACKGROUND OF THE INVENTION \n 1. Field of t...,1. A keyboard apparatus comprising:\n a suppor...,"[keyboard, apparatus, is, including, plurality...","{'keyboard': 3, 'apparatus': 1, 'is': 3, 'incl..."
1,US-7556524-B2,"An easy-pull type swivel plug includes a body,...",BACKGROUND OF THE INVENTION \n 1. Field of t...,"1. An easy-pull type swivel plug, comprising:\...","[an, easy, pull, type, swivel, plug, includes,...","{'an': 1, 'easy': 1, 'pull': 1, 'type': 1, 'sw..."
2,US-7338315-B2,The invention relates to a closure device comp...,FIELD OF THE INVENTION \n The invention rela...,1. A closure device comprising:\n a wall havin...,"[the, invention, relates, to, closure, device,...","{'the': 16, 'invention': 2, 'relates': 1, 'to'..."
3,US-6843642-B2,An air compressor with shock-absorption rubber...,BACKGROUND OF THE INVENTION \n (a) Field of ...,1. An air compressor with shock-absorption rub...,"[an, air, compressor, with, shock, absorption,...","{'an': 2, 'air': 7, 'compressor': 7, 'with': 2..."
4,US-9433212-B2,Provided is a novel plant growth regulator. Th...,TECHNICAL FIELD \n The present invention r...,The invention claimed is: \n \n 1. A...,"[provided, is, novel, plant, growth, regulator...","{'provided': 1, 'is': 1, 'novel': 1, 'plant': ..."
5,US-5536436-A,A liquid laundry detergent composition contain...,FIELD OF THE INVENTION \n The present inve...,What is claimed is: \n \n 1. A heavy...,"[liquid, laundry, detergent, composition, cont...","{'liquid': 2, 'laundry': 1, 'detergent': 2, 'c..."
6,US-2015111807-A1,A liquid laundry detergent composition compris...,FIELD OF THE INVENTION \n The present inve...,What is claimed is: \n \n 1 . A li...,"[liquid, laundry, detergent, composition, comp...","{'liquid': 1, 'laundry': 1, 'detergent': 1, 'c..."
7,US-7605322-B2,As a player inputs a performance of a music pi...,TECHNICAL FIELD \n The present invention rel...,1. An apparatus for automatically starting an ...,"[as, player, inputs, performance, of, music, p...","{'as': 2, 'player': 2, 'inputs': 1, 'performan..."
8,US-7205268-B2,A low-foaming aqueous liquid laundry detergent...,FIELD OF THE INVENTION \n The present inve...,1. A low-foaming aqueous liquid laundry deterg...,"[low, foaming, aqueous, liquid, laundry, deter...","{'low': 2, 'foaming': 1, 'aqueous': 1, 'liquid..."
9,US-6910186-B2,A new class of avatars (“organizational avatar...,CROSS-REFERENCE TO APPENDICES ATTACHED HERETO ...,"1. A method of communicating between users, th...","[new, class, of, avatars, organizational, avat...","{'new': 1, 'class': 1, 'of': 7, 'avatars': 9, ..."


Let's try to implement Jaccard_Score similarity</br>
I have to first create set representations of BOW instead of dictionary representation

In [9]:
def setOfWordize(tokenized_abstract):
  a = set()
  a.update(tokenized_abstract)
  return a

dataframe['Set_Representation_Of_Tokenized_Abstract'] = dataframe['Tokenized_Abstract'].apply(setOfWordize)

In [10]:
dataframe

,Publication_Number,Abstract,Description,Claim,Tokenized_Abstract,Bag_Of_Words_Abstract,Set_Representation_Of_Tokenized_Abstract
0,US-7365259-B2,A keyboard apparatus is including a plurality ...,BACKGROUND OF THE INVENTION \n 1. Field of t...,1. A keyboard apparatus comprising:\n a suppor...,"[keyboard, apparatus, is, including, plurality...","{'keyboard': 3, 'apparatus': 1, 'is': 3, 'incl...","{the, portion, under, lowest, is, including, t..."
1,US-7556524-B2,"An easy-pull type swivel plug includes a body,...",BACKGROUND OF THE INVENTION \n 1. Field of t...,"1. An easy-pull type swivel plug, comprising:\...","[an, easy, pull, type, swivel, plug, includes,...","{'an': 1, 'easy': 1, 'pull': 1, 'type': 1, 'sw...","{the, corresponds, pull, rotatable, liftable, ..."
2,US-7338315-B2,The invention relates to a closure device comp...,FIELD OF THE INVENTION \n The invention rela...,1. A closure device comprising:\n a wall havin...,"[the, invention, relates, to, closure, device,...","{'the': 16, 'invention': 2, 'relates': 1, 'to'...","{the, device, closed, which, open, flap, an, t..."
3,US-6843642-B2,An air compressor with shock-absorption rubber...,BACKGROUND OF THE INVENTION \n (a) Field of ...,1. An air compressor with shock-absorption rub...,"[an, air, compressor, with, shock, absorption,...","{'an': 2, 'air': 7, 'compressor': 7, 'with': 2...","{actuated, the, is, an, bottom, plate, of, to,..."
4,US-9433212-B2,Provided is a novel plant growth regulator. Th...,TECHNICAL FIELD \n The present invention r...,The invention claimed is: \n \n 1. A...,"[provided, is, novel, plant, growth, regulator...","{'provided': 1, 'is': 1, 'novel': 1, 'plant': ...","{as, the, regulator, is, provided, active, pla..."
5,US-5536436-A,A liquid laundry detergent composition contain...,FIELD OF THE INVENTION \n The present inve...,What is claimed is: \n \n 1. A heavy...,"[liquid, laundry, detergent, composition, cont...","{'liquid': 2, 'laundry': 1, 'detergent': 2, 'c...","{selected, the, using, is, active, about, furt..."
6,US-2015111807-A1,A liquid laundry detergent composition compris...,FIELD OF THE INVENTION \n The present inve...,What is claimed is: \n \n 1 . A li...,"[liquid, laundry, detergent, composition, comp...","{'liquid': 1, 'laundry': 1, 'detergent': 1, 'c...","{selected, the, two, an, three, of, consisting..."
7,US-7605322-B2,As a player inputs a performance of a music pi...,TECHNICAL FIELD \n The present invention rel...,1. An apparatus for automatically starting an ...,"[as, player, inputs, performance, of, music, p...","{'as': 2, 'player': 2, 'inputs': 1, 'performan...","{start, the, corresponds, under, run, player, ..."
8,US-7205268-B2,A low-foaming aqueous liquid laundry detergent...,FIELD OF THE INVENTION \n The present inve...,1. A low-foaming aqueous liquid laundry deterg...,"[low, foaming, aqueous, liquid, laundry, deter...","{'low': 2, 'foaming': 1, 'aqueous': 1, 'liquid...","{aqueous, the, surfactant, machines, about, of..."
9,US-6910186-B2,A new class of avatars (“organizational avatar...,CROSS-REFERENCE TO APPENDICES ATTACHED HERETO ...,"1. A method of communicating between users, th...","[new, class, of, avatars, organizational, avat...","{'new': 1, 'class': 1, 'of': 7, 'avatars': 9, ...","{limited, using, background, is, eliminating, ..."


Now - Jaccard_Similarity Function</br>
(I couldnt figure this out with scikit)

In [11]:
a = dataframe.at[8,'Set_Representation_Of_Tokenized_Abstract']
b = dataframe.at[5,'Set_Representation_Of_Tokenized_Abstract']

In [12]:
def jaccard_similarity(list1, list2):
  intersectionVar = len(list(set(list1).intersection(list2)))
  unionVar = (len(list1)) + len(list2) - intersectionVar
  return float(intersectionVar) / unionVar

This is the Jaccard Similarity for two like patents

In [13]:
jaccard_similarity(a,b)

0.20512820512820512

Two not similar patents

In [14]:
d = dataframe.at[9,'Set_Representation_Of_Tokenized_Abstract']
c = dataframe.at[1,'Set_Representation_Of_Tokenized_Abstract']
jaccard_similarity(c,d)

0.07333333333333333

See? Much lower than .2

# Before I move on -<br>
I have to vectorize my bag of words,<br>
which involves creating a corpus/vocab of words

In [ ]:
def create_corpus(datafr):
	corpus = []

	for i in datafr.index:
		token_set = datafr['Set_Representation_Of_Tokenized_Abstract'][i]
		corpus_set = set(corpus)
		new_tokens = token_set - corpus_set
		corpus = corpus + list(new_tokens)

	return corpus

myCorpus = create_corpus(dataframe)
myCorpus


### Now to vectorize -
In order to vectorize,<br> I have to create a function that takes each patent's 'Tokenized_Abstract' and compare it to the corpus. <br>
Then do .apply

In [18]:
def vectorize(tokenized_abstract):
  vector = []
  for word in myCorpus:
    vector.append(tokenized_abstract.count(word))
  return vector


In [19]:
dataframe['Vector_Representation_Of_Tokenized_Abstract'] = dataframe['Tokenized_Abstract'].apply(vectorize)

In [20]:
dataframe

,Publication_Number,Abstract,Description,Claim,Tokenized_Abstract,Bag_Of_Words_Abstract,Set_Representation_Of_Tokenized_Abstract,Vector_Representation_Of_Tokenized_Abstract
0,US-7365259-B2,A keyboard apparatus is including a plurality ...,BACKGROUND OF THE INVENTION \n 1. Field of t...,1. A keyboard apparatus comprising:\n a suppor...,"[keyboard, apparatus, is, including, plurality...","{'keyboard': 3, 'apparatus': 1, 'is': 3, 'incl...","{the, portion, under, lowest, is, including, t...","[12, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 9, 1, 1, 1,..."
1,US-7556524-B2,"An easy-pull type swivel plug includes a body,...",BACKGROUND OF THE INVENTION \n 1. Field of t...,"1. An easy-pull type swivel plug, comprising:\...","[an, easy, pull, type, swivel, plug, includes,...","{'an': 1, 'easy': 1, 'pull': 1, 'type': 1, 'sw...","{the, corresponds, pull, rotatable, liftable, ...","[17, 0, 0, 0, 3, 0, 2, 0, 0, 1, 0, 4, 0, 3, 0,..."
2,US-7338315-B2,The invention relates to a closure device comp...,FIELD OF THE INVENTION \n The invention rela...,1. A closure device comprising:\n a wall havin...,"[the, invention, relates, to, closure, device,...","{'the': 16, 'invention': 2, 'relates': 1, 'to'...","{the, device, closed, which, open, flap, an, t...","[16, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 7, 0, 4, 1,..."
3,US-6843642-B2,An air compressor with shock-absorption rubber...,BACKGROUND OF THE INVENTION \n (a) Field of ...,1. An air compressor with shock-absorption rub...,"[an, air, compressor, with, shock, absorption,...","{'an': 2, 'air': 7, 'compressor': 7, 'with': 2...","{actuated, the, is, an, bottom, plate, of, to,...","[9, 0, 0, 0, 4, 0, 0, 0, 0, 2, 0, 3, 0, 2, 0, ..."
4,US-9433212-B2,Provided is a novel plant growth regulator. Th...,TECHNICAL FIELD \n The present invention r...,The invention claimed is: \n \n 1. A...,"[provided, is, novel, plant, growth, regulator...","{'provided': 1, 'is': 1, 'novel': 1, 'plant': ...","{as, the, regulator, is, provided, active, pla...","[1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
5,US-5536436-A,A liquid laundry detergent composition contain...,FIELD OF THE INVENTION \n The present inve...,What is claimed is: \n \n 1. A heavy...,"[liquid, laundry, detergent, composition, cont...","{'liquid': 2, 'laundry': 1, 'detergent': 2, 'c...","{selected, the, using, is, active, about, furt...","[4, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 3, 0, 1, 0, ..."
6,US-2015111807-A1,A liquid laundry detergent composition compris...,FIELD OF THE INVENTION \n The present inve...,What is claimed is: \n \n 1 . A li...,"[liquid, laundry, detergent, composition, comp...","{'liquid': 1, 'laundry': 1, 'detergent': 1, 'c...","{selected, the, two, an, three, of, consisting...","[1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 1, 0, ..."
7,US-7605322-B2,As a player inputs a performance of a music pi...,TECHNICAL FIELD \n The present invention rel...,1. An apparatus for automatically starting an ...,"[as, player, inputs, performance, of, music, p...","{'as': 2, 'player': 2, 'inputs': 1, 'performan...","{start, the, corresponds, under, run, player, ...","[13, 0, 1, 0, 0, 0, 0, 0, 0, 3, 0, 6, 0, 4, 0,..."
8,US-7205268-B2,A low-foaming aqueous liquid laundry detergent...,FIELD OF THE INVENTION \n The present inve...,1. A low-foaming aqueous liquid laundry deterg...,"[low, foaming, aqueous, liquid, laundry, deter...","{'low': 2, 'foaming': 1, 'aqueous': 1, 'liquid...","{aqueous, the, surfactant, machines, about, of...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, ..."
9,US-6910186-B2,A new class of avatars (“organizational avatar...,CROSS-REFERENCE TO APPENDICES ATTACHED HERETO ...,"1. A method of communicating between users, th...","[new, class, of, avatars, organizational, avat...","{'new': 1, 'class': 1, 'of': 7, 'avatars': 9, ...","{limited, using, background, is, eliminating, ...","[12, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 7, 0, 5, 0,..."


## And Finally -<br>
Now that I've created vectors, I can finally use numpy and sklearn (scikit)

In [22]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as cs

#Thank you Zach!
#scikit cosine similarity accepts 2d arrays as parameters,
# that is why need to reshape it 
def cosine_similarity_func(vectorA, vectorB):
  v1 = np.array(vectorA).reshape(1,-1)
  v2 = np.array(vectorB).reshape(1,-1)
  return cs(v1, v2)[0][0]



vec1= (dataframe['Vector_Representation_Of_Tokenized_Abstract'][0])
vec2= (dataframe['Vector_Representation_Of_Tokenized_Abstract'][6])

cosine_similarity_func(vec1, vec2)

0.2255237862381975

Now to figure out how to create a table representing both the Jaccard similarities<br>
and cosine similarities across patents